In [3]:
'Run some regression analyses within the still conditions'



import os
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from joblib import Parallel, delayed
from collections import defaultdict
from matplotlib.cm import get_cmap
from scipy.ndimage import gaussian_filter1d
from scipy.stats import circstd

def train_test_split(epochs, test_chunk, target_id, cv_id='session_nr'):
    epochs_test = epochs[epochs.metadata[cv_id] == test_chunk]
    epochs_train = epochs[epochs.metadata[cv_id] != test_chunk]
    X_test = epochs_test._data
    X_train = epochs_train._data

    y_train = epochs_train.metadata[target_id].to_numpy()
    y_test = epochs_test.metadata[target_id].to_numpy()

    return X_train, X_test, y_train, y_test

def linear_regression_timegen_window(X_train, X_test, y_train_deg, model_class, model_kwargs, t_train_window, t_test):
    y_train_sin = np.sin(np.deg2rad(y_train_deg))
    y_train_cos = np.cos(np.deg2rad(y_train_deg))
    X_train_avg = X_train[:, :, t_train_window[0]:t_train_window[1]].mean(axis=2)

    pipe_sin = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', model_class(**model_kwargs))
    ])
    pipe_cos = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', model_class(**model_kwargs))
    ])

    pipe_sin.fit(X_train_avg, y_train_sin)
    pipe_cos.fit(X_train_avg, y_train_cos)

    y_pred_sin = pipe_sin.predict(X_test[:, :, t_test])
    y_pred_cos = pipe_cos.predict(X_test[:, :, t_test])

    y_pred_angle = np.rad2deg(np.arctan2(y_pred_sin, y_pred_cos)) % 360
    return y_pred_angle

def run_regression(epochs_train, cv_id, model_class, model_kwargs, t_train_window):
    all_y_pred = []
    all_y_true = []

    for test_chunk in np.unique(epochs_train.metadata[cv_id]):
        print(f'running cv #{test_chunk} out of {len(np.unique(epochs_train.metadata[cv_id]))}')

        X_train, X_test, y_train, y_test = train_test_split(
            epochs=epochs_train,
            test_chunk=test_chunk,
            target_id='degrees',
            cv_id=cv_id
        )
        y_predicted = Parallel(n_jobs=8)(
            delayed(linear_regression_timegen_window)(
                X_train, X_test, y_train,
                model_class, model_kwargs,
                t_train_window=t_train_window,
                t_test=t
            ) for t in range(X_test.shape[2])
        )

        all_y_pred.append(y_predicted)
        all_y_true.append(y_test)

    return np.array(all_y_pred), np.array(all_y_true)

def get_train_window(peak_sample, window_size=30):
    start = max(0, peak_sample - window_size)
    end = peak_sample + window_size + 1
    return (start, end)

def circular_distance(pred, true):
    return np.abs(((pred - true + 180) % 360) - 180)

def signed_circular_distance(pred, true):
    return ((pred - true + 180) % 360) - 180

def resultant_vector_length(errors_deg):
    radians = np.deg2rad(errors_deg)
    return np.abs(np.mean(np.exp(1j * radians)))

def compute_circular_error_metrics(errors_deg):
    errors_rad = np.deg2rad(errors_deg)
    circ_std = np.rad2deg(circstd(errors_rad, high=np.pi, low=-np.pi))
    resultant_length = np.abs(np.mean(np.exp(1j * errors_rad)))
    return circ_std, resultant_length

subjects = [f"S{i:02}" for i in range(1, 16)]
bids_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
data_path = f'{bids_dir}/derivatives/preprocessed/'

regressors = {
    'Ridge 1000': (Ridge, {'alpha': 1000})
}

csv_path = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/LDA-16way Static/Mean/Peak_Times.csv'
peak_df = pd.read_csv(csv_path, index_col='Subject')

all_subject_errors = defaultdict(list)
output_dir = "/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/WithinStillRegression"
os.makedirs(output_dir, exist_ok=True)

all_condition_errors = defaultdict(list)
group_circ_stds = defaultdict(list)
group_resultant_lengths = defaultdict(list)
group_unsigned_errors = defaultdict(list)
for Subject in subjects:
    print(f"Processing {Subject}")
    fn_still = f'sub-{Subject}_Still_preprocessed-epo.fif'
    epochs_still = mne.read_epochs(data_path + fn_still)

    epochs_still.metadata['degrees_string'] = [k.split('/')[-1] for k in epochs_still.metadata['trial_type']]
    epochs_still = epochs_still[epochs_still.metadata['degrees_string'] != 'catch']
    epochs_still.metadata['degrees'] = [int(i) for i in epochs_still.metadata['degrees_string']]
    epochs_still.metadata.rename(columns={'run_nr': 'run'}, inplace=True)

    peak1_sample = int(peak_df.loc['all', 'peak1_sample'])
    t_train_window = get_train_window(peak1_sample, window_size=30)

    model_predictions = {}
    for reg_name, (model_class, model_kwargs) in regressors.items():
        y_pred, y_true = run_regression(epochs_still, 'run', model_class, model_kwargs, t_train_window)
        predicted_angles = np.transpose(y_pred, (0, 2, 1))
        model_predictions[reg_name] = predicted_angles

        np.save(os.path.join(output_dir, f"{Subject}_{reg_name}_predictions.npy"), predicted_angles)
        np.save(os.path.join(output_dir, f"{Subject}_{reg_name}_true.npy"), y_true)

    metadata = epochs_still.metadata
    all_conditions = sorted(set(metadata['degrees_string']))
    colors = get_cmap('viridis', len(all_conditions))

    for reg_name, predicted_angles in model_predictions.items():
        condition_signed_errors = {}
        condition_unsigned_errors = {}
        condition_metrics = {}

        plt.figure(figsize=(12, 6))
        for condition in all_conditions:
            condition_angle = int(condition)
            condition_mask = metadata['degrees_string'].values == condition
            condition_mask = condition_mask.reshape(4, 240)

            pred = predicted_angles[condition_mask]
            pred = pred.reshape(-1, predicted_angles.shape[2])

            signed_errors = ((pred - condition_angle + 180) % 360) - 180
            condition_signed_errors[condition_angle] = signed_errors

            unsigned_errors = np.abs(signed_errors)
            condition_unsigned_errors[condition_angle] = unsigned_errors
            all_condition_errors[condition_angle].append(unsigned_errors)

            circ_stds, result_lengths = zip(*[
                compute_circular_error_metrics(signed_errors[:, t])
                for t in range(signed_errors.shape[1])
            ])

            circ_stds = np.array(circ_stds)
            result_lengths = np.array(result_lengths)
            condition_metrics[condition_angle] = {
                'circ_std': circ_stds,
                'resultant_length': result_lengths
            }
            group_circ_stds[condition_angle].append(circ_stds)
            group_resultant_lengths[condition_angle].append(result_lengths)
            group_unsigned_errors[condition_angle].append(unsigned_errors)

            plt.plot(gaussian_filter1d(circ_stds, 2), label=f'{condition_angle}°', linewidth=1, alpha=0.75)

        mean_errors = np.array([condition_metrics[cond]['circ_std'] for cond in sorted(condition_metrics.keys())])
        plt.plot(gaussian_filter1d(mean_errors.mean(axis=0), 2), label='Mean', color='black', linewidth=2, alpha = .85)

        plt.title(f'{Subject} – Circular Std of Signed Error by Condition ({reg_name})')
        plt.xlabel('Timepoints')
        plt.ylabel('Circular Std (deg)')
        plt.legend(ncol=4, fontsize=8)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{Subject}_{reg_name}_AllConditions_CircularStd.png'))
        plt.close()

        plt.figure(figsize=(12, 6))
        for condition_angle, metrics in condition_metrics.items():
            plt.plot(gaussian_filter1d(metrics['resultant_length'], 2), label=f'{condition_angle}°', linewidth=1, alpha=0.75)
        all_r = np.array([metrics['resultant_length'] for metrics in condition_metrics.values()])
        plt.plot(gaussian_filter1d(all_r.mean(axis=0), 2), label='Mean', color='black', linewidth=2, alpha = .85)

        plt.title(f'{Subject} – Resultant Vector Length by Condition ({reg_name})')
        plt.xlabel('Timepoints')
        plt.ylabel('Resultant Vector Length (R)')
        plt.ylim(0, 1.05)
        plt.legend(ncol=4, fontsize=8)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{Subject}_{reg_name}_AllConditions_ResultantLength.png'))
        plt.close()

        plt.figure(figsize=(12, 6))
        for condition_angle, errors in condition_unsigned_errors.items():
            plt.plot(gaussian_filter1d(errors.mean(axis=0), 2), label=f'{condition_angle}°', linewidth=1, alpha=0.75)
        all_mae = np.array([errors.mean(axis=0) for errors in condition_unsigned_errors.values()])
        plt.plot(gaussian_filter1d(all_mae.mean(axis=0), 2), label='Mean', color='black', linewidth=2, alpha = .85)

        plt.title(f'{Subject} – Unsigned Circular Error by Condition ({reg_name})')
        plt.xlabel('Timepoints')
        plt.ylabel('Mean Absolute Error (deg)')
        plt.legend(ncol=4, fontsize=8)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{Subject}_{reg_name}_AllConditions_UnsignedError.png'))
        plt.close()

        np.save(os.path.join(output_dir, f"{Subject}_{reg_name}_signed_errors.npy"), condition_signed_errors)
        np.save(os.path.join(output_dir, f"{Subject}_{reg_name}_unsigned_errors.npy"), condition_unsigned_errors)
        np.save(os.path.join(output_dir, f"{Subject}_{reg_name}_error_metrics.npy"), condition_metrics)

# === GROUP LEVEL AVERAGING AND PLOTTING ===
print("Computing group-level averages...")

for metric_name, metric_dict in [
    ('CircularStd', group_circ_stds),
    ('ResultantLength', group_resultant_lengths),
    ('UnsignedError', group_unsigned_errors)
]:
    plt.figure(figsize=(12, 6))
    for condition_angle in sorted(metric_dict.keys()):
        all_data = np.array(metric_dict[condition_angle])
        mean_metric = all_data.mean(axis=0)
        smoothed = gaussian_filter1d(mean_metric, 2)
        plt.plot(smoothed, label=f'{condition_angle}°', linewidth=1, alpha=0.75)

    # Grand average across all conditions
    all_means = [np.array(metric_dict[cond]) for cond in metric_dict]
    stacked = np.vstack(all_means)
    overall_mean = stacked.mean(axis=0)
    smoothed_mean = gaussian_filter1d(overall_mean, 2)
    plt.plot(smoothed_mean, label='Grand Mean', color='black', linewidth=2)

    plt.title(f'Group-Level {metric_name} by Condition')
    plt.xlabel('Timepoints')
    ylabel = {
        'CircularStd': 'Circular Std (deg)',
        'ResultantLength': 'Resultant Vector Length (R)',
        'UnsignedError': 'Mean Absolute Error (deg)'
    }[metric_name]
    plt.ylabel(ylabel)
    if metric_name == 'ResultantLength':
        plt.ylim(0, 1.05)
    plt.grid(True)
    plt.legend(ncol=4, fontsize=8)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'GroupAverage_{metric_name}_AllConditions.png'))
    plt.close()



Processing S01
Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Still_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
1104 matching events found
No baseline correction applied
0 projection items activated
running cv #0 out of 4


Process LokyProcess-2:
Process LokyProcess-7:
Traceback (most recent call last):
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py", line 481, in _process_worker
    if time() - _last_memory_leak_check > _MEMORY_LEAK_CHECK_DELAY:
       ^^^^^^
KeyboardInterrupt
Traceback (most recent call last):
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12

KeyboardInterrupt: 

In [17]:
'producing std/vector length'



import os
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from joblib import Parallel, delayed
from collections import defaultdict
from matplotlib.cm import get_cmap
from scipy.ndimage import gaussian_filter1d
from scipy.stats import circstd

def train_test_split(epochs, test_chunk, target_id, cv_id='session_nr'):
    epochs_test = epochs[epochs.metadata[cv_id] == test_chunk]
    epochs_train = epochs[epochs.metadata[cv_id] != test_chunk]
    X_test = epochs_test._data
    X_train = epochs_train._data

    y_train = epochs_train.metadata[target_id].to_numpy()
    y_test = epochs_test.metadata[target_id].to_numpy()

    return X_train, X_test, y_train, y_test

def linear_regression_timegen_window(X_train, X_test, y_train_deg, model_class, model_kwargs, t_train_window, t_test):
    y_train_sin = np.sin(np.deg2rad(y_train_deg))
    y_train_cos = np.cos(np.deg2rad(y_train_deg))
    X_train_avg = X_train[:, :, t_train_window[0]:t_train_window[1]].mean(axis=2)

    pipe_sin = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', model_class(**model_kwargs))
    ])
    pipe_cos = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', model_class(**model_kwargs))
    ])

    pipe_sin.fit(X_train_avg, y_train_sin)
    pipe_cos.fit(X_train_avg, y_train_cos)

    y_pred_sin = pipe_sin.predict(X_test[:, :, t_test])
    y_pred_cos = pipe_cos.predict(X_test[:, :, t_test])

    y_pred_angle = np.rad2deg(np.arctan2(y_pred_sin, y_pred_cos)) % 360
    return y_pred_angle

def run_regression(epochs_train, cv_id, model_class, model_kwargs, t_train_window):
    all_y_pred = []
    all_y_true = []

    for test_chunk in np.unique(epochs_train.metadata[cv_id]):
        print(f'running cv #{test_chunk} out of {len(np.unique(epochs_train.metadata[cv_id]))}')

        X_train, X_test, y_train, y_test = train_test_split(
            epochs=epochs_train,
            test_chunk=test_chunk,
            target_id='degrees',
            cv_id=cv_id
        )
        y_predicted = Parallel(n_jobs=8)(
            delayed(linear_regression_timegen_window)(
                X_train, X_test, y_train,
                model_class, model_kwargs,
                t_train_window=t_train_window,
                t_test=t
            ) for t in range(X_test.shape[2])
        )

        all_y_pred.append(y_predicted)
        all_y_true.append(y_test)

    return np.array(all_y_pred), np.array(all_y_true)

def get_train_window(peak_sample, window_size=30):
    start = max(0, peak_sample - window_size)
    end = peak_sample + window_size + 1
    return (start, end)

def circular_distance(pred, true):
    return np.abs(((pred - true + 180) % 360) - 180)

def signed_circular_distance(pred, true):
    return ((pred - true + 180) % 360) - 180

def resultant_vector_length(errors_deg):
    radians = np.deg2rad(errors_deg)
    return np.abs(np.mean(np.exp(1j * radians)))

def compute_circular_error_metrics(errors_deg):
    errors_rad = np.deg2rad(errors_deg)
    circ_std = np.rad2deg(circstd(errors_rad, high=np.pi, low=-np.pi))
    resultant_length = np.abs(np.mean(np.exp(1j * errors_rad)))
    return circ_std, resultant_length

subjects = [f"S{i:02}" for i in range(1, 16)]
bids_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
data_path = f'{bids_dir}/derivatives/preprocessed/'

regressors = {
    'Ridge 1000': (Ridge, {'alpha': 1000})
}

csv_path = "/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Plots/BlenderExperiment/AllSubjects/WithinStill/ShrutiMay-22/SVM/AllSubjects_PeakTimes.csv"
peak_df = pd.read_csv(csv_path, index_col='Subject')

all_subject_errors = defaultdict(list)
output_dir = "/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Plots/Jun2/Late"
os.makedirs(output_dir, exist_ok=True)

all_condition_errors = defaultdict(list)
group_circ_stds = defaultdict(list)
group_resultant_lengths = defaultdict(list)
group_unsigned_errors = defaultdict(list)
for Subject in subjects:
    print(f"Processing {Subject}")
    fn_still = f'sub-{Subject}_Still_preprocessed-epo.fif'
    epochs_still = mne.read_epochs(data_path + fn_still)

    epochs_still.metadata['degrees_string'] = [k.split('/')[-1] for k in epochs_still.metadata['trial_type']]
    epochs_still = epochs_still[epochs_still.metadata['degrees_string'] != 'catch']
    epochs_still.metadata['degrees'] = [int(i) for i in epochs_still.metadata['degrees_string']]
    epochs_still.metadata.rename(columns={'run_nr': 'run'}, inplace=True)

    peak1_sample = int(peak_df.loc[Subject, 'peak2_sample_plus240'])
    t_train_window = get_train_window(peak1_sample, window_size=30)

    model_predictions = {}
    for reg_name, (model_class, model_kwargs) in regressors.items():
        y_pred, y_true = run_regression(epochs_still, 'run', model_class, model_kwargs, t_train_window)
        predicted_angles = np.transpose(y_pred, (0, 2, 1))
        model_predictions[reg_name] = predicted_angles

        np.save(os.path.join(output_dir + f"/data/{Subject}_{reg_name}_predictions.npy"), predicted_angles)
        np.save(os.path.join(output_dir + f"/data/{Subject}_{reg_name}_true.npy"), y_true)

    metadata = epochs_still.metadata
    all_conditions = sorted(set(metadata['degrees_string']))
    colors = get_cmap('viridis', len(all_conditions))

    for reg_name, predicted_angles in model_predictions.items():
        condition_signed_errors = {}
        condition_unsigned_errors = {}
        condition_metrics = {}

        plt.figure(figsize=(12, 6))
        for condition in all_conditions:
            condition_angle = int(condition)
            condition_mask = metadata['degrees_string'].values == condition
            condition_mask = condition_mask.reshape(4, 240)

            pred = predicted_angles[condition_mask]
            pred = pred.reshape(-1, predicted_angles.shape[2])

            signed_errors = ((pred - condition_angle + 180) % 360) - 180
            condition_signed_errors[condition_angle] = signed_errors

            unsigned_errors = np.abs(signed_errors)
            condition_unsigned_errors[condition_angle] = unsigned_errors
            all_condition_errors[condition_angle].append(unsigned_errors)

            circ_stds, result_lengths = zip(*[
                compute_circular_error_metrics(signed_errors[:, t])
                for t in range(signed_errors.shape[1])
            ])

            circ_stds = np.array(circ_stds)
            result_lengths = np.array(result_lengths)
            condition_metrics[condition_angle] = {
                'circ_std': circ_stds,
                'resultant_length': result_lengths
            }
            group_circ_stds[condition_angle].append(circ_stds)
            group_resultant_lengths[condition_angle].append(result_lengths)
            group_unsigned_errors[condition_angle].append(unsigned_errors)

        # PLOT SIGNED ERROR WITH STANDARD DEVIATION
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # Plot signed error on primary y-axis
        for condition_angle, signed_errors in condition_signed_errors.items():
            mean_signed = np.mean(signed_errors, axis=0)
            std_signed = np.std(signed_errors, axis=0)

            # Smooth both
            mean_signed_smooth = gaussian_filter1d(mean_signed, 2)
            std_signed_smooth = gaussian_filter1d(std_signed, 2)

            ax1.plot(mean_signed_smooth, label=f'{condition_angle}°', linewidth=1.5, alpha=0.8)
            ax1.fill_between(np.arange(len(mean_signed_smooth)),
                            mean_signed_smooth - std_signed_smooth,
                            mean_signed_smooth + std_signed_smooth,
                            alpha=0.25)

        # Mean across conditions
        all_signed = np.array([signed for signed in condition_signed_errors.values()])
        mean_across_conditions = np.mean(all_signed, axis=(0, 1))
        std_across_conditions = np.std(all_signed, axis=(0, 1))

        mean_across_conditions = gaussian_filter1d(mean_across_conditions, 2)
        std_across_conditions = gaussian_filter1d(std_across_conditions, 2)
        '''
        ax1.plot(mean_across_conditions, label='Mean Across Conditions', color='black', linewidth=2)
        ax1.fill_between(np.arange(len(mean_across_conditions)),
                        mean_across_conditions - std_across_conditions,
                        mean_across_conditions + std_across_conditions,
                        color='gray', alpha=0.3)
        '''
        # Customize primary axis
        ax1.set_xlabel('Timepoints')
        ax1.set_ylabel('Signed Error (deg)', color='tab:blue')
        ax1.axhline(0, color='k', linestyle='--', linewidth=1)
        ax1.tick_params(axis='y', labelcolor='tab:blue')

        # Add second y-axis for normalized std
        ax2 = ax1.twinx()
        for condition_angle, signed_errors in condition_signed_errors.items():
            std_signed = np.std(signed_errors, axis=0)
            std_smooth = gaussian_filter1d(std_signed, 2)

            # Normalize std for better visibility
            std_norm = (std_smooth - np.min(std_smooth)) / (np.max(std_smooth) - np.min(std_smooth))

            ax2.plot(std_norm, linestyle='--', alpha=0.3, label=f'{condition_angle}° Std')

        ax2.set_ylabel('Normalized Std Dev (arbitrary units)', color='tab:red')
        ax2.tick_params(axis='y', labelcolor='tab:red')

        # Combined legend
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        fig.legend(lines1 + lines2, labels1 + labels2, loc='upper right', ncol=4, fontsize=8)

        plt.title(f'{Subject} – Mean Signed Error with Variability ({reg_name})')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{Subject}_{reg_name}_SignedError_with_Variability.png'))
        plt.close()




        np.save(os.path.join(output_dir, f"{Subject}_{reg_name}_signed_errors.npy"), condition_signed_errors)
        np.save(os.path.join(output_dir, f"{Subject}_{reg_name}_error_metrics.npy"), condition_metrics)

# === GROUP LEVEL AVERAGING AND PLOTTING ===
print("Computing group-level averages...")

# === GROUP-LEVEL PLOT: Signed error per condition averaged across subjects ===

plt.figure(figsize=(12, 6))
for condition_angle, subject_errors in all_condition_errors.items():
    # subject_errors: list of arrays [n_trials x n_timepoints] per subject
    all_errors = np.concatenate(subject_errors, axis=0)  # shape: (n_total_trials, n_timepoints)

    # Compute mean and std across all trials (across all subjects)
    mean_signed = np.mean(all_errors, axis=0)
    std_signed = np.std(all_errors, axis=0)

    # Smooth
    mean_signed_smooth = gaussian_filter1d(mean_signed, 2)
    std_signed_smooth = gaussian_filter1d(std_signed, 2)

    plt.plot(mean_signed_smooth, label=f'{condition_angle}°', linewidth=1.5, alpha=0.8)
    plt.fill_between(np.arange(len(mean_signed_smooth)),
                     mean_signed_smooth - std_signed_smooth,
                     mean_signed_smooth + std_signed_smooth,
                     alpha=0.25)

# Overall mean across all conditions
all_errors_stack = np.concatenate([np.concatenate(errs, axis=0) for errs in all_condition_errors.values()], axis=0)
mean_all = np.mean(all_errors_stack, axis=0)
std_all = np.std(all_errors_stack, axis=0)

mean_all_smooth = gaussian_filter1d(mean_all, 2)
std_all_smooth = gaussian_filter1d(std_all, 2)

plt.plot(mean_all_smooth, label='Mean Across Conditions', color='black', linewidth=2)
plt.fill_between(np.arange(len(mean_all_smooth)),
                 mean_all_smooth - std_all_smooth,
                 mean_all_smooth + std_all_smooth,
                 color='gray', alpha=0.3)

plt.title('Group Average – Mean Signed Error by Condition')
plt.xlabel('Timepoints')
plt.ylabel('Signed Error (deg)')
plt.axhline(0, color='k', linestyle='--', linewidth=1)
plt.legend(ncol=4, fontsize=8)
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'Group_Mean_SignedError_AllConditions.png'))
plt.close()



Processing S01
Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Still_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
1104 matching events found
No baseline correction applied
0 projection items activated
running cv #0 out of 4
running cv #1 out of 4
running cv #2 out of 4
running cv #3 out of 4


/var/folders/9m/6b08zp213znc3k12g6gpjj9rqx2ly9/T/ipykernel_94183/777387201.py:144: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = get_cmap('viridis', len(all_conditions))


Processing S02
Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S02_Still_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
1104 matching events found
No baseline correction applied
0 projection items activated
running cv #0 out of 4


Process LokyProcess-95:
Traceback (most recent call last):
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/montesinossl/anaconda3/envs/mne2/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py", line 481, in _process_worker
    if time() - _last_memory_leak_check > _MEMORY_LEAK_CHECK_DELAY:
       ^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

<Figure size 1200x600 with 0 Axes>